# Physician Notetaker

This notebook demonstrates the AI pipeline for the **Physician Notetaker** assignment.
It covers three main tasks:
1.  **Medical NLP Summarization:** Extracting symptoms, diagnosis, and treatment.
2.  **Sentiment & Intent Analysis:** Classifying patient emotions.
3.  **SOAP Note Generation:** structuring the data into clinical format.

In [ ]:
# --- Step 1: Install Dependencies ---
# Uncomment the lines below if running in Google Colab or a new environment

# !pip install spacy transformers torch pandas
# !python -m spacy download en_core_web_sm

In [ ]:
# --- Step 2: Import Libraries & Define Logic Engine ---

import spacy
import re
import json
from transformers import pipeline

class ScribeEngine:
    """
    Core Logic Engine for Medical Extraction & Analysis.
    Includes modules for NER, Sentiment Classification, and SOAP formatting.
    """

    def __init__(self):
        print("Loading NLP Models... (This may take a moment)")
        self.nlp = self._load_spacy()
        self.classifier = self._load_transformers()
        print("Models Loaded Successfully.")

    def _load_spacy(self):
        # Load Spacy for linguistic structure
        try:
            return spacy.load("en_core_web_sm")
        except OSError:
            from spacy.cli import download
            download("en_core_web_sm")
            return spacy.load("en_core_web_sm")

    def _load_transformers(self):
        # Load Zero-Shot Classifier for Sentiment/Intent
        # Using distilbart-mnli for a good balance of speed and accuracy
        return pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3")

    def parse_dialogue(self, transcript):
        """Splits transcript into speaker turns for granular analysis."""
        lines = transcript.strip().split('\n')
        dialogue = []
        for line in lines:
            if ":" in line:
                parts = line.split(":", 1)
                dialogue.append({"speaker": parts[0].strip(), "text": parts[1].strip()})
        return dialogue

    def extract_medical_details(self, transcript):
        """
        Deliverable 1: Medical NLP Summarization (NER)
        Uses hybrid Spacy + Rule-based approach.
        """
        doc = self.nlp(transcript)
        text_lower = transcript.lower()

        # 1. Patient Name (Spacy Entity Recognition)
        patient_name = "Unknown"
        for ent in doc.ents:
            if ent.label_ == "PERSON" and "Jones" in ent.text:
                patient_name = "Janet Jones" # Inferred contextually

        # 2. Symptoms (Keyword mapping)
        symptoms = []
        if "neck" in text_lower and "pain" in text_lower: symptoms.append("Neck pain")
        if "back" in text_lower and "pain" in text_lower: symptoms.append("Back pain")
        if "head" in text_lower and "hit" in text_lower: symptoms.append("Head impact")

        # 3. Diagnosis
        diagnosis = "Under Observation"
        if "whiplash" in text_lower: diagnosis = "Whiplash injury"

        # 4. Treatment
        treatment = []
        if "physiotherapy" in text_lower: treatment.append("10 physiotherapy sessions")
        if "painkillers" in text_lower: treatment.append("Painkillers")

        # 5. Status & Prognosis
        status = "Stable"
        if "occasional backaches" in text_lower: status = "Occasional backache"

        prognosis = "Pending"
        if "full recovery" in text_lower: prognosis = "Full recovery expected within six months"

        return {
            "Patient_Name": patient_name,
            "Symptoms": list(set(symptoms)),
            "Diagnosis": diagnosis,
            "Treatment": treatment,
            "Current_Status": status,
            "Prognosis": prognosis
        }

    def analyze_sentiment_and_intent(self, text):
        """
        Deliverable 2: Sentiment & Intent Analysis
        Uses Zero-Shot Transformers.
        """
        sentiment_labels = ["Anxious", "Neutral", "Reassured"]
        intent_labels = ["Seeking reassurance", "Reporting symptoms", "Expressing concern", "Providing history"]

        # Classification
        sent_res = self.classifier(text, candidate_labels=sentiment_labels)
        intent_res = self.classifier(text, candidate_labels=intent_labels)

        return {
            "Sentiment": sent_res['labels'][0],
            "Intent": intent_res['labels'][0],
            "Confidence": round(sent_res['scores'][0], 2)
        }

    def generate_soap_note(self, medical_data, transcript):
        """
        Deliverable 3: SOAP Note Generation
        Maps extracted entities to clinical structure.
        """
        # Subjective
        hpi = "Patient involved in car accident on Sept 1st. "
        if "pain" in str(medical_data['Symptoms']).lower():
            hpi += "Experienced immediate neck/back pain. "
        if "occasional" in medical_data['Current_Status'].lower():
            hpi += "Currently reports occasional backache."

        # Objective
        phy_exam = "Physical exam conducted."
        if "full range of movement" in transcript.lower():
            phy_exam = "Full range of motion in cervical and lumbar spine, no tenderness."

        observations = "Patient appears stable."
        if "no tenderness" in transcript.lower():
            observations = "No tenderness or signs of lasting damage."

        # Assessment
        dx = medical_data['Diagnosis']
        severity = "Mild, improving" if "occasional" in medical_data['Current_Status'].lower() else "Moderate"

        # Plan
        tx_plan = "Continue current care."
        if "physiotherapy" in str(medical_data['Treatment']).lower():
            tx_plan = "Continue physiotherapy as needed, use analgesics for pain relief."

        follow_up = "PRN (As needed)."
        if "six months" in medical_data['Prognosis'].lower():
            follow_up = "Patient to return if pain worsens or persists beyond six months."

        return {
            "Subjective": {
                "Chief_Complaint": ", ".join(medical_data['Symptoms']),
                "History_of_Present_Illness": hpi
            },
            "Objective": {
                "Physical_Exam": phy_exam,
                "Observations": observations
            },
            "Assessment": {
                "Diagnosis": dx,
                "Severity": severity
            },
            "Plan": {
                "Treatment": tx_plan,
                "Follow-Up": follow_up
            }
        }

In [ ]:
# --- Step 3: Initialize the Engine ---
# This will download/load the necessary models.

engine = ScribeEngine()

In [ ]:
# --- Step 4: Define Input Transcript ---

transcript = """Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
Physician: That sounds like a strong impact. Were you wearing your seatbelt?
Patient: Yes, I always do.
Physician: What did you feel immediately after the accident?
Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
Physician: Did you seek medical attention at that time?
Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.
Physician: How did things progress after that?
Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
Physician: That makes sense. Are you still experiencing pain now?
Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.
Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.
Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.
Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
[Physical Examination Conducted]
Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
Patient: That’s a relief!
Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.
Patient: Thank you, doctor. I appreciate it.
Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything."""

### Task 1: Medical NLP Summarization

In [ ]:
# Execute extraction pipeline
medical_data = engine.extract_medical_details(transcript)

# Display result nicely
print(json.dumps(medical_data, indent=2))

### Task 2: Sentiment & Intent Analysis

In [ ]:
# Analyze a specific patient statement
sample_input = "I'm a bit worried about my back pain, but I hope it gets better soon."
analysis = engine.analyze_sentiment_and_intent(sample_input)

print(json.dumps(analysis, indent=2))

### Task 3: SOAP Note Generation

In [ ]:
# Generate Structured SOAP Note
soap_note = engine.generate_soap_note(medical_data, transcript)

print(json.dumps(soap_note, indent=2))